In [47]:
import numpy as np
import torch
import torch.nn as nn
import os
import datetime as dt
from datetime import timedelta
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

In [48]:
device = torch.device('cpu')

In [49]:
class InputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, u_vert, u_hori, rho):
        # use feq as input field
        # todo: make this layer be also learnable
        
        # feq(x,v) = C[v] * rho * (1 + 3 v.u + 4.5 (v.u)^2 - 1.5 u^2)
        feq = torch.empty((3, 3, self.height, self.width), dtype=torch.float32, device=device)
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        u_squared = u_vert ** 2 + u_hori ** 2
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + 3.0 * v_dot_u + 4.5 * v_dot_u * v_dot_u - 1.5 * u_squared)
                
        return feq

In [50]:
# test for InputLayer
# # Later, I'll check the immutability of the InputLayer
#  - it must put out the same tensor even after the weights are updated
input_layer_u_vert = torch.tensor([[0.1]], dtype=torch.float32)
input_layer_u_hori = torch.tensor([[-0.3]], dtype=torch.float32)
input_layer_rho = torch.tensor([[0.8]], dtype=torch.float32)

input_layer = InputLayer(1, 1).to(device)
feq = input_layer(input_layer_u_vert, input_layer_u_hori, input_layer_rho)
print(torch.sum(feq[2,:,0,0] - feq[0,:,0,0]) / torch.sum(feq[:,:,0,0])) # u_vert
print(torch.sum(feq[:,2,0,0] - feq[:,0,0,0]) / torch.sum(feq[:,:,0,0])) # u_hori
print(torch.sum(feq[:,:,0,0])) # rho

tensor(0.1000)
tensor(-0.3000)
tensor(0.8000)


In [51]:
class StreamingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w0 = nn.Parameter(torch.zeros((in_height-2, in_width-2), dtype=torch.float32, device=device))
        self.w1 = nn.Parameter(torch.ones((in_height-2, in_width-2), dtype=torch.float32, device=device))
        
    def forward(self, f_prev):
        # f(x,v) = w0(x,v) + w1(x,v) * f_prev(x-v,v)
        
        f = torch.empty((3, 3, self.in_height-2, self.in_width-2), dtype=torch.float32, device=device)
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                f[dr+1,dc+1,:,:] = self.w0 + self.w1 * f_prev[dr+1,dc+1,1-dr:self.in_height-1-dr,1-dc:self.in_width-1-dc]
        
        return f
                

In [52]:
# test for StreamingLayer
input_f_prev = torch.zeros((3, 3, 3, 3), dtype=torch.float64)
input_f_prev[2,2,0,0] = 1.0
input_f_prev[2,1,0,1] = 2.0
input_f_prev[2,0,0,2] = 3.0
input_f_prev[1,2,1,0] = 4.0
input_f_prev[1,1,1,1] = 5.0
input_f_prev[1,0,1,2] = 6.0
input_f_prev[0,2,2,0] = 7.0
input_f_prev[0,1,2,1] = 8.0
input_f_prev[0,0,2,2] = 9.0
print(input_f_prev)

streaming_layer = StreamingLayer(3, 3).to(device)
f_streamed = streaming_layer(input_f_prev)
print(f_streamed)

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 9.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 8., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [7., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 6.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 5., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [4., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 3.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 2., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[1., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]], dtype=torch.float64)
tensor([[[[9.]],

         [[8.]],

         [[7.]]],


        [[[6.]],

         [[5.]],

         [[4.]]],


        [[[3.]],

         [[2.]],

         [[1.]]]], grad_fn=<CopySlices>)


In [53]:
class CollidingLayer(nn.Module):
    def __init__(self, in_height, in_width):
        super().__init__()
        self.in_height = in_height
        self.in_width = in_width
        
        self.w1 = nn.Parameter(torch.full((in_height, in_width), fill_value=3.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w2 = nn.Parameter(torch.full((in_height, in_width), fill_value=0.0, dtype=torch.float32, device=device), requires_grad=True)
        self.w3 = nn.Parameter(torch.full((in_height, in_width), fill_value=4.5, dtype=torch.float32, device=device), requires_grad=True)
        self.w4 = nn.Parameter(torch.full((in_height, in_width), fill_value=-1.5, dtype=torch.float32, device=device), requires_grad=True)
        
    def forward(self, f_prev):
        # feq(x,v) = C(v) * rho(x) * (1 + w1(x,v) v.u(x) + w2(x,v) vXu(x) + w3(x,v) (v.u(x))^2 + w4(x,v) u(x)^2)
        # f = (f_prev + feq) / 2
        C = torch.tensor([[1.0/36.0, 1.0/9.0, 1.0/36.0], [1.0/9.0, 4.0/9.0, 1.0/9.0], [1.0/36.0, 1.0/9.0, 1.0/36.0]], dtype=torch.float32, device=device)
        
        rho = torch.sum(f_prev, dim=(0,1))
        u_vert = (f_prev[2,0,:,:] + f_prev[2,1,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[0,1,:,:] - f_prev[0,2,:,:]) / rho
        u_hori = (f_prev[0,2,:,:] + f_prev[1,2,:,:] + f_prev[2,2,:,:] - f_prev[0,0,:,:] - f_prev[1,0,:,:] - f_prev[2,0,:,:]) / rho
        u_squared = u_vert ** 2 + u_hori ** 2
        feq = torch.empty((3, 3, self.in_height, self.in_width), dtype=torch.float32, device=device)
        
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                v_dot_u = dr * u_vert + dc * u_hori
                vXu = dr * u_hori - dc * u_vert
                feq[dr+1, dc+1, :, :] = C[dr+1,dc+1] * rho * (1.0 + (self.w1 + self.w3 * v_dot_u) * v_dot_u + self.w2 * vXu + self.w4 * u_squared)
        
        return (f_prev + feq) / 2.0

In [54]:
# test for CollidingLayer
input_f = torch.tensor([[[[9., 9.]],[[8., 8.]],[[7., 7.]]],[[[6., 6.]],[[5., 5.]],[[4., 4.]]],[[[3., 3.]],[[2., 2.]],[[1., 1.]]]], dtype=torch.float32)
colliding_layer = CollidingLayer(1, 2)
collided_field = colliding_layer(input_f)

print(torch.sum(collided_field[:,:,0,0])) # rho 45.0
print((torch.sum(collided_field[2,:,0,0]) - torch.sum(collided_field[0,:,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_vert
print((1.+2.+3.-9.-8.-7.)/45.)
print((torch.sum(collided_field[:,2,0,0]) - torch.sum(collided_field[:,0,0,0])) / torch.sum(collided_field[:,:,0,0])) # u_hori
print((1.+4.+7.-3.-6.-9.)/45.)

tensor(45., grad_fn=<SumBackward0>)
tensor(-0.4000, grad_fn=<DivBackward0>)
-0.4
tensor(-0.1333, grad_fn=<DivBackward0>)
-0.13333333333333333


In [55]:
class OutputLayer(nn.Module):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        
    def forward(self, f):
        rho = torch.sum(f, dim=(0,1))
        u_vert = (f[2,0,:,:] + f[2,1,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[0,1,:,:] - f[0,2,:,:]) / rho
        u_hori = (f[0,2,:,:] + f[1,2,:,:] + f[2,2,:,:] - f[0,0,:,:] - f[1,0,:,:] - f[2,0,:,:]) / rho
        return u_vert, u_hori

In [56]:
class LBM(nn.Module):
    margin = 3
    
    def __init__(self, height, width):
        super(LBM, self).__init__()
        self.input_layer = InputLayer(height, width)
        self.streaming1_layer = StreamingLayer(height, width)
        self.colliding1_layer = CollidingLayer(height-2, width-2)
        self.streaming2_layer = StreamingLayer(height-2, width-2)
        self.colliding2_layer = CollidingLayer(height-4, width-4)
        self.streaming3_layer = StreamingLayer(height-4, width-4)
        self.output_layer = OutputLayer(height-6, width-6)
        
    def forward(self, u_vert, u_hori, rho):
        f = self.input_layer(u_vert, u_hori, rho)
        f = self.streaming1_layer(f)
        f = self.colliding1_layer(f)
        f = self.streaming2_layer(f)
        f = self.colliding2_layer(f)
        f = self.streaming3_layer(f)
        u_vert_out, u_hori_out = self.output_layer(f)
        return u_vert_out, u_hori_out

In [57]:
def loss_func(u_vert_pred, u_hori_pred, u_vert_target, u_hori_target):
    # MSE
    return torch.sum(torch.abs(u_vert_pred - u_vert_target) + torch.abs(u_hori_pred - u_hori_target))

In [58]:
class WeatherDataset(torch.utils.data.Dataset):
    def __init__(self, src_dir, datetimes, transform, target_transform, margin):
        self.src_dir = src_dir
        self.u_vert_paths = []
        self.u_hori_paths = []
        self.rho_paths = []
        self.u_vert_target_paths = []
        self.u_hori_target_paths = []
        self.margin = margin # this value is the number of streaming layers that reduce the border cells.
        # eg. ...                          xxx
        #     ...  --(Streaming Layer)-->  x.x
        #     ...                          xxx
        for datetime in datetimes:
            self.u_vert_paths.append(os.path.join(src_dir, datetime.strftime("u_vert_%Y%m%d%H.npy")))
            self.u_hori_paths.append(os.path.join(src_dir, datetime.strftime("u_hori_%Y%m%d%H.npy")))
            self.rho_paths.append(os.path.join(src_dir, datetime.strftime("pressure_%Y%m%d%H.npy")))
            datetime += timedelta(hours=3)
            self.u_vert_target_paths.append(os.path.join(src_dir, datetime.strftime('u_vert_%Y%m%d%H.npy')))
            self.u_hori_target_paths.append(os.path.join(src_dir, datetime.strftime('u_hori_%Y%m%d%H.npy')))
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.u_vert_paths)
    
    def __getitem__(self, idx):
        u_vert = torch.from_numpy(np.load(self.u_vert_paths[idx]))
        u_hori = torch.from_numpy(np.load(self.u_hori_paths[idx]))
        rho = torch.from_numpy(np.load(self.rho_paths[idx]))
        u_vert_target = torch.from_numpy(np.load(self.u_vert_target_paths[idx]))
        u_hori_target = torch.from_numpy(np.load(self.u_hori_target_paths[idx]))
        u_vert, u_hori, rho = self.transform(u_vert, u_hori, rho)
        u_vert_target, u_hori_target = self.target_transform(u_vert_target, u_hori_target, self.margin)
        return {
            "u_vert": u_vert,
            "u_hori": u_hori,
            "rho": rho,
            "u_vert_target": u_vert_target,
            "u_hori_target": u_hori_target
        }

In [59]:
def transform(u_vert, u_hori, rho):
    # 下向き正
    return u_vert * -0.01, u_hori * 0.01, rho * 0.0001

def target_transform(u_vert, u_hori, margin):
    if margin == 0:
        return u_vert * -0.01, u_hori * 0.01
    else:
        return u_vert[margin:-margin, margin:-margin] * -0.01, u_hori[margin:-margin, margin:-margin] * 0.01

In [60]:
# test for WeatherDataset
datetimeset = [dt.datetime(2011, 7, 1), dt.datetime(2011, 7, 2)]
src_dir = '../lab_data/npy'
weather_dataset = WeatherDataset(src_dir, datetimeset, transform, target_transform, 1)
u_vert_direct = torch.from_numpy(np.load('../lab_data/npy/u_vert_2011070103.npy'))
u_vert_target = weather_dataset.__getitem__(0)['u_vert_target']
print(weather_dataset.__len__()) # 2
print(torch.equal(u_vert_target, u_vert_direct[1:-1, 1:-1] * -0.01)) # True
print(u_vert_direct.size())

2
True
torch.Size([505, 481])


In [61]:
# todo: enable the model to receive batch data
def main():
    print('Creating a dataset and dataloader...')
    data_dir = '../lab_data/npy'
    datetimeset = [dt.datetime(2011+i, 7, 1, 0) + timedelta(days=j) for i in range(10) for j in range(92)]
    dataset = WeatherDataset(data_dir, datetimeset, transform, target_transform, LBM.margin)
    batch_size = 1
    train_split = 0.8
    shuffle_dataset = True # if false, this model will learn only by the 'past' data, and forcast the 'future' data.
    random_seed = 42
    
    dataset_size =  len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(train_split * dataset_size))
    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, valid_indices = indices[:split], indices[split:]
    
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(valid_indices)
    
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
    valid_sampler = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=4)
    
    print('Creating the model...')
    lbm = LBM(505, 481).to(device) # got by the prev cell
    
    num_epochs = 10
    learning_rate = 0.1
    
    optimizer = torch.optim.Adam(lbm.parameters(), lr=learning_rate)
    
    print("\nbat_size = %3d " % batch_size)
    print("optimizer = " + str(optimizer))
    print("max_epochs = %3d " % num_epochs)
    print("lrn_rate = %0.3f " % learning_rate)
    
    print('Training the model...')
    lbm.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        
        for (batch_idx, batch) in enumerate(train_loader):
            optimizer.zero_grad()
            u_vert_out, u_hori_out = lbm(batch['u_vert'].squeeze(), batch['u_hori'].squeeze(), batch['rho'].squeeze())
            loss_val = loss_func(u_vert_out, u_hori_out, batch['u_vert_target'].squeeze(), batch['u_hori_target'].squeeze())
            epoch_loss += loss_val.item()
            loss_val.backward()
            if batch_idx % 50 == 0:
                print(f'batch: {batch_idx}')
                print(f'loss: {loss_val.item()}')
            optimizer.step()
            
        print("epoch = %4d   loss = %0.4f" % (epoch, epoch_loss))
    print('Done')
    
    print('Computing the model accuracy')
    lbm.eval()
        
main()

Creating a dataset and dataloader...
Creating the model...

bat_size =   1 
optimizer = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.1
    maximize: False
    weight_decay: 0
)
max_epochs =  10 
lrn_rate = 0.100 
Training the model...
batch: 0
loss: 4366.02197265625
batch: 50
loss: 6154.4404296875
batch: 100
loss: 6478.8896484375
batch: 150
loss: 6058.3896484375
batch: 200
loss: 5819.2470703125
batch: 250
loss: 6781.2138671875
batch: 300
loss: 5816.2353515625
batch: 350
loss: 5709.02197265625
batch: 400
loss: 5237.18408203125
batch: 450
loss: 5901.3203125
batch: 500
loss: 6972.65771484375
batch: 550
loss: 6125.12451171875
batch: 600
loss: 5593.2314453125
batch: 650
loss: 5190.61328125
batch: 700
loss: 6838.43701171875
epoch =    0   loss = 4560034.5342
batch: 0
loss: 5358.20263671875
batch: 50
loss: 4392.283203125
batch: 100
loss: 5157.0712890625
batch: 150